# TensorFlow 기본 원리

+ (1) 그래프 준비하기 (construction)
+ (2) 그래프 실행하기 (execution)

---

# MNIST 기본 세팅

**필요한 모듈 임포트**

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

**tf 버전 확인**

In [3]:
tf.__version__

'1.3.0'

**mnist 데이터 다운로드**

In [43]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [44]:
print(len(mnist.train.images[0])) # 784
print(len(mnist.train.labels[0])) # 10

784
10


---

# 1) Softmax classifier

## Part I. 그래프 그리기

+ 계산 그래프 그림

![](../images/c-mnist-tensorflow_1.jpg)

![](../images/c-mnist-tensorflow_2.jpg)

**00 random seed**

In [45]:
tf.set_random_seed(777)  # reproducibility

**01 placeholder**

In [46]:
X = tf.placeholder(dtype=tf.float32, shape=[None, 784])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

**02 predictions**

In [52]:
W = tf.Variable(tf.random_normal(shape=[784, 10]))
b = tf.Variable(tf.random_normal(shape=[10]))

predictions = tf.matmul(X, W) + b

# 2) Neural Networks

## Part I. 그래프 그리기

+ 계산 그래프 그림

![](../images/c-mnist-tensorflow_3.jpg)

**02 predictions**

In [59]:
W1 = tf.Variable(tf.random_normal(shape=[784, 256]))
b1 = tf.Variable(tf.random_normal(shape=[256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([256, 256]))
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.Variable(tf.random_normal([256, 10]))
b3 = tf.Variable(tf.random_normal([10]))

predictions = tf.matmul(L2, W3) + b3

---

**03 cost function and optimizer**

In [63]:
cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

**04 accuracy**

In [61]:
is_correct = tf.equal(tf.argmax(predictions , axis=1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

## Part II. 그래프 실행

**05 training epoch/batch**

In [64]:
epochs = 15
batch_size = 100


sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys})
        avg_cost += c / total_batch

    print("Epoch:", "%04d" % (epoch + 1), "cost =", "{:.9f}".format(avg_cost))

Epoch: 0001 cost = 49.361731147
Epoch: 0002 cost = 9.433782093
Epoch: 0003 cost = 5.525125694
Epoch: 0004 cost = 3.335861625
Epoch: 0005 cost = 2.966776492
Epoch: 0006 cost = 2.338150602
Epoch: 0007 cost = 2.076080219
Epoch: 0008 cost = 2.016731138
Epoch: 0009 cost = 1.699481401
Epoch: 0010 cost = 1.267029428
Epoch: 0011 cost = 1.429122764
Epoch: 0012 cost = 1.277773317
Epoch: 0013 cost = 1.068471093
Epoch: 0014 cost = 0.911328619
Epoch: 0015 cost = 0.904147109


**06 test accuracy**

In [65]:
print("Accuracy:", sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Accuracy: 0.9663


---

## 연습장 

+ `tf.set_random_seed(777)`의 의미

In [66]:
tf.set_random_seed(777)

In [67]:
a = tf.random_uniform([1])
b = tf.random_normal([1])

In [68]:
with tf.Session() as sess1:
    print(sess1.run(a))  # generates 'A1'
    print(sess1.run(a))  # generates 'A2'
    print(sess1.run(b))  # generates 'B1'
    print(sess1.run(b))  # generates 'B2'

[ 0.56603217]
[ 0.91976988]
[ 0.50993961]
[ 1.28496039]


In [69]:
with tf.Session() as sess2:
    print(sess2.run(a))  # generates 'A1'
    print(sess2.run(a))  # generates 'A2'
    print(sess2.run(b))  # generates 'B1'
    print(sess2.run(b))  # generates 'B2'

[ 0.56603217]
[ 0.91976988]
[ 0.50993961]
[ 1.28496039]


---

## 연습장

+ cost 구할 때 tf.reduce_mean을 해줘야 하는 이유

In [70]:
cost_before = tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=Y)

In [71]:
cost_before_val = sess.run(cost_before, feed_dict={X: mnist.test.images, Y: mnist.test.labels})

In [74]:
cost = tf.reduce_mean(cost_before)

In [75]:
cost_val = sess.run(cost, feed_dict={X: mnist.test.images, Y: mnist.test.labels})

In [76]:
cost_val

3.1060178

In [77]:
cost_val.shape

()

In [78]:
cost_before_val.shape

(10000,)